In [1]:
import sys
import json
import os
import tensorflow as tf
import numpy as np
#import tensorflow_probability as tfp
import numba as nb
import scipy.io
import matplotlib.pyplot as plt
import optuna
# Manually making sure the numpy random seeds are "the same" on all devices


# LOCAL IMPORTS

eqnPath = 'DataGenerator'
sys.path.append(eqnPath)
#sys.path.append("utils")
from logger import Logger
#from neuralnetwork_opt import NeuralNetwork
from burgersutil import prep_data, plot_inf_cont_results

C:\Users\SZF\anaconda3\envs\GPUtensorflow\lib\_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [2]:
import tensorflow as tf
import numpy as np
%run ./TheFmodelforPINN_KFR.ipynb

from custom_lbfgs import lbfgs, Struct


class NeuralNetwork(object):
    def __init__(self, opt_hp, logger, X_f, ub, lb, alpha, beta, amp, n_layers, neurons,\
                 model, optimizer, x_lab=None, wn=None, sign=None):

        architecture = [neurons for i in range(n_layers+2)] #+2 to accommodate input and output
        architecture[0] = 2 #Inputs
        architecture[-1] = 1 #Outputs
        layers = architecture
        
        # Setting up the optimizers with the hyper-parameters
        self.nt_config = Struct()
        self.nt_config.learningRate = opt_hp["nt_lr"]
        self.nt_config.maxIter = opt_hp["nt_epochs"]
        self.nt_config.nCorrection = opt_hp["nt_ncorr"]
        self.nt_config.tolFun = 1.0 * np.finfo(float).eps
        self.tf_epochs = opt_hp["tf_epochs"]
        self.dtype = "float64"
        self.model = model
        self.tf_optimizer = optimizer
        self.alpha = alpha
        self.beta = beta
        self.amp = amp
        self.x_lab = x_lab
        self.wn = wn
        self.sign = sign

        # Separating the collocation coordinates
        self.x_f = self.tensor(X_f[:, 0:1]) #[-1,1] random points
        self.t_f = self.tensor(X_f[:, 1:2]) #[0,1] random points

#     # Computing the sizes of weights/biases for future decomposition
        self.sizes_w = []
        self.sizes_b = []
        for i, width in enumerate(layers):
            if i != 1:
                self.sizes_w.append(int(width * layers[1]))
                self.sizes_b.append(int(width if i != 0 else layers[1]))

        self.logger = logger

    # Defining custom loss
    # @tf.function
    #def loss(self, u, u_pred):
        #return tf.reduce_mean(tf.square(u - u_pred))

    def summary(self):
        return self.model.summary()

    def tensor(self, X):
        return tf.convert_to_tensor(X, dtype=self.dtype)
    
    # @tf.function  
    # Defining custom loss
    def loss(self, u, u_pred):
        f_pred = self.f_model(self.alpha, self.beta, self.amp, self.x_lab, self.wn, self.sign)
        return tf.reduce_mean(tf.square(u - u_pred)) + tf.reduce_mean(tf.square(f_pred))
#######################################################
    def source(self, x, us, alpha, beta, amp):
        ainv = 8 * sqrt(pi * beta) * (1 + scipy.special.erf(1 / sqrt(4 * beta)))
        source = np.exp(-(x - xi(us, alpha, amp)) ** 2 / (4 * beta)) / ainv
        return source

    def flux(self, Y, us, amp, x_lab, wn, sign):
        D = Dshock(us, amp, x_lab, wn, sign)
        return 0.5 * Y ** 2. - D * Y
#######################################################
    # The actual PINN
    def f_model(self, alpha, beta, amp, x_lab, wn, sign):
        # Using the new GradientTape paradigm of TF2.0,
        # which keeps track of operations to get the gradient at runtime
        # We need the boundary conditions from the data matrix of u as well

        with tf.GradientTape(persistent=True) as tape:
            # Watching the two inputs we’ll need later, x and t
            tape.watch(self.x_f)
            tape.watch(self.t_f)
            # Packing together the inputs
            bc = np.zeros_like(self.x_f) #To get the predicted value of us at all times i.e. getting u_s
            X_f = tf.stack([self.x_f[:, 0], self.t_f[:, 0]], axis=1)
            BC_f = tf.stack([bc[:,0], self.t_f[:, 0]], axis=1) #predicted value of u at boundry at all times i.e. getting u_s

            # Getting the prediction
            u = self.model(X_f)
            us = self.model(BC_f)

            f =  self.flux(u, us, amp, x_lab, wn, sign)
            # Deriving INSIDE the tape (since we’ll need the x derivative of this later, u_xx)
            f_x = tape.gradient(f, self.x_f)

        s = self.source(self.x_f,us,alpha,beta,amp)    
        u_t = tape.gradient(u, self.t_f)

    # Letting the tape go
        del tape
    # Buidling the PINNs
        return u_t+f_x-s

    def grad(self, X, u):
        with tf.GradientTape() as tape:
            loss_value = self.loss(u, self.model(X))
        grads = tape.gradient(loss_value, self.wrap_training_variables())
        return loss_value, grads

    def wrap_training_variables(self):
        var = self.model.trainable_variables
        return var

    def get_params(self, numpy=False):
        return []

    def get_weights(self, convert_to_tensor=True):
        w = []
        for layer in self.model.layers[1:]:
            weights_biases = layer.get_weights()
            weights = weights_biases[0].flatten()
            biases = weights_biases[1]
            w.extend(weights)
            w.extend(biases)
        if convert_to_tensor:
            w = self.tensor(w)
        return w

    def set_weights(self, w):
        for i, layer in enumerate(self.model.layers[1:]):
            start_weights = sum(self.sizes_w[:i]) + sum(self.sizes_b[:i])
            end_weights = sum(self.sizes_w[:i+1]) + sum(self.sizes_b[:i])
            weights = w[start_weights:end_weights]
            w_div = int(self.sizes_w[i] / self.sizes_b[i])
            weights = tf.reshape(weights, [w_div, self.sizes_b[i]])
            biases = w[end_weights:end_weights + self.sizes_b[i]]
            weights_biases = [weights, biases]
            layer.set_weights(weights_biases)

    def get_loss_and_flat_grad(self, X, u):
        def loss_and_flat_grad(w):
            with tf.GradientTape() as tape:
                self.set_weights(w)
                loss_value = self.loss(u, self.model(X))
            grad = tape.gradient(loss_value, self.wrap_training_variables())
            grad_flat = []
            for g in grad:
                grad_flat.append(tf.reshape(g, [-1]))
            grad_flat = tf.concat(grad_flat, 0)
            return loss_value, grad_flat

        return loss_and_flat_grad

    def tf_optimization(self, X_u, u):
        self.logger.log_train_opt("Adam")
        for epoch in range(self.tf_epochs):
            loss_value = self.tf_optimization_step(X_u, u)
            self.logger.log_train_epoch(epoch, loss_value)

    # @tf.function
    def tf_optimization_step(self, X_u, u):
        loss_value, grads = self.grad(X_u, u)
        self.tf_optimizer.apply_gradients(
                zip(grads, self.wrap_training_variables()))
        return loss_value

    def nt_optimization(self, X_u, u):
        self.logger.log_train_opt("LBFGS")
        loss_and_flat_grad = self.get_loss_and_flat_grad(X_u, u)
    # tfp.optimizer.lbfgs_minimize(
    #   loss_and_flat_grad,
    #   initial_position=self.get_weights(),
    #   num_correction_pairs=nt_config.nCorrection,
    #   max_iterations=nt_config.maxIter,
    #   f_relative_tolerance=nt_config.tolFun,
    #   tolerance=nt_config.tolFun,
    #   parallel_iterations=6)
        self.nt_optimization_steps(loss_and_flat_grad)


    def nt_optimization_steps(self, loss_and_flat_grad):
        lbfgs(loss_and_flat_grad,
              self.get_weights(),
              self.nt_config, Struct(), True,
              lambda epoch, loss, is_iter:
              self.logger.log_train_epoch(epoch, loss, "", is_iter))

    def fit(self, X_u, u, X_star, u_star):
        self.logger.log_train_start(self)

        # Creating the tensors
        X_u = self.tensor(X_u)
        u = self.tensor(u)

        # Optimizing
        self.tf_optimization(X_u, u)
        self.nt_optimization(X_u, u)

        #self.logger.log_train_end(self.tf_epochs + self.nt_config.maxIter)

    def predict(self, X_star):
        u_star = self.model(X_star)
        f_star = self.f_model(alpha, beta, amp, x_lab, wn, sign)
        return u_star.numpy(), f_star.numpy()

In [16]:
def create_optimizer(trial):
    # We optimize the choice of optimizers as well as their parameters.
    kwargs = {}
    optimizer_options = "Adam"
    #optimizer_selected = trial.suggest_categorical("optimizer", optimizer_options)
    kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 0.019, 0.02, log=True)
    kwargs["beta_1"] = 0.99
    optimizer = getattr(tf.optimizers, optimizer_options)(**kwargs)
    return optimizer

def create_model(trial, lb, ub):
    dtype = "float64"
    # Descriptive Keras model
    tf.keras.backend.set_floatx(dtype)

    #n_layers = trial.suggest_int("n_layers", 5, 12)
    n_layers = 9
    #neurons  =  trial.suggest_int("neurons", 35, 45)
    neurons = 45
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(2,)))
    model.add(tf.keras.layers.Lambda(
        lambda X: 2.0*(X - lb)/(ub - lb) - 1.0))
    for i in range(n_layers):
        width = trial.suggest_int("n_units_l{}".format(i), neurons, neurons, log=True)
        model.add(tf.keras.layers.Dense(
            width, activation=tf.nn.tanh,
            kernel_initializer="glorot_normal"))
    model.add(tf.keras.layers.Dense(
            1, activation=None,
            kernel_initializer="glorot_normal"))
    return model, n_layers, neurons

def params(trial):
    nt_lr = trial.suggest_float("nt_lr", 0.25, 0.40)
    #nt_lr = 0.5
    #nt_ncorr = trial.suggest_int("nt_ncorr", 50, 120)
    nt_ncorr = 100
    nt_epochs = trial.suggest_int("nt_epochs", 150,300)
    #nt_epochs = 400
    tf_epochs = trial.suggest_int("tf_epochs", 150,250)
    #tf_epochs = 450
    #N_u = trial.suggest_int("N_u", 2000,5000)
    N_u = 1000
    #N_f = trial.suggest_int("N_f", 13500,16000)
    N_f = 14522
    log_freq = 10
    hp = {
        'nt_lr': nt_lr,
        'nt_ncorr': nt_ncorr,
        'nt_epochs': nt_epochs,
        'tf_epochs': tf_epochs,
        'log_frequency': log_freq,
        'N_u': N_u,
        'N_f': N_f
    }
    return hp

In [17]:
np.random.seed(1234)
tf.random.set_seed(1234)

alpha=4.7
beta=0.1
amp=0.0
x_lab=None 
wn=None 
sign=None

def objective(trial):
    np.random.seed(1234)
    tf.random.set_seed(1234)
    class KFRInformedNN(NeuralNetwork):
        def __init__(self, opt_hp, logger, X_f, ub, lb, alpha, beta, amp, n_layers, neurons, model, optimizer, x_lab=None, wn=None, sign=None):
            super().__init__(opt_hp, logger, X_f, ub, lb, alpha, beta, amp, n_layers, neurons, model, optimizer, x_lab=None, wn=None, sign=None)


    eqnPath = 'DataGenerator'
    path = os.path.join(eqnPath, 'amp_0alpha_4.7')
    opt_hp = params(trial)
    x, t, X, T, Exact_u, X_star, u_star, \
        X_u_train, u_train, X_f, ub, lb = prep_data(
            path, opt_hp['N_u'], opt_hp['N_f'], noise=0.0)
    


    model, n_layers, neurons = create_model(trial, lb, ub)
    optimizer = create_optimizer(trial)
#     nt_lr = nt_learn(trial)
#     nt_ncorr = nt_ncorrection(trial)
    logger = Logger(opt_hp)
    pinn = KFRInformedNN(opt_hp, logger, X_f, ub, lb, alpha ,beta, amp, n_layers, neurons, model, optimizer)
#     summ = pinn.summary()
#     print(summ)
    
    def learn(model, optimizer, X, u, X_star, u_star): 
        return pinn.fit(X_u_train, u_train, X_star, u_star)
    def test(model, optimizer, X, u, X_star, u_star):
            u_pred, _ = pinn.predict(X_star)
            error = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
            return error
        
    for _ in range(1):
        learn(model, optimizer, X_u_train, u_train, X_star, u_star)
        #error_value = test(model, optimizer, X_u_train, u_train, X_star, u_star)
        #print(error_value)
    error_value = test(model, optimizer, X_u_train, u_train, X_star, u_star)
    return -error_value

In [18]:
import time

start_time = time.time()
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=10, interval_steps=2
    ))
study.optimize(objective, n_trials=15)

print("Number of finished trials: ", len(study.trials))



print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

print("--- %s seconds ---" % (time.time() - start_time))

[I 2021-08-04 06:55:23,532] A new study created in memory with name: no-name-d9f4fbcf-b53c-474c-96a6-3eeaa35d5ece


Hyperparameters:
{
  "nt_lr": 0.2646534304852239,
  "nt_ncorr": 100,
  "nt_epochs": 270,
  "tf_epochs": 209,
  "log_frequency": 10,
  "N_u": 1000,
  "N_f": 14522
}

TensorFlow version: 2.3.0
Eager execution: True
GPU-accerelated: True

Training started
-- Starting Adam optimization --
tf_epoch =      0  elapsed = 00:00 (+00.2)  loss = 6.9438e-01  
tf_epoch =     10  elapsed = 00:01 (+01.6)  loss = 1.4771e+00  
tf_epoch =     20  elapsed = 00:03 (+01.6)  loss = 8.9169e-02  
tf_epoch =     30  elapsed = 00:05 (+01.5)  loss = 2.0784e-02  
tf_epoch =     40  elapsed = 00:06 (+01.6)  loss = 6.2586e-02  
tf_epoch =     50  elapsed = 00:08 (+01.6)  loss = 6.5468e-02  
tf_epoch =     60  elapsed = 00:10 (+01.6)  loss = 4.4643e-02  
tf_epoch =     70  elapsed = 00:11 (+01.6)  loss = 5.8189e-02  
tf_epoch =     80  elapsed = 00:13 (+01.6)  loss = 3.0288e-02  
tf_epoch =     90  elapsed = 00:14 (+01.6)  loss = 3.4708e-02  
tf_epoch =    100  elapsed = 00:16 (+01.6)  loss = 3.1287e-02  
tf_epoch =

[I 2021-08-04 06:57:04,482] Trial 0 finished with value: -0.08271048691230308 and parameters: {'nt_lr': 0.2646534304852239, 'nt_epochs': 270, 'tf_epochs': 209, 'n_units_l0': 45, 'n_units_l1': 45, 'n_units_l2': 45, 'n_units_l3': 45, 'n_units_l4': 45, 'n_units_l5': 45, 'n_units_l6': 45, 'n_units_l7': 45, 'n_units_l8': 45, 'adam_learning_rate': 0.019093991835594685}. Best is trial 0 with value: -0.08271048691230308.


Hyperparameters:
{
  "nt_lr": 0.3083755998718507,
  "nt_ncorr": 100,
  "nt_epochs": 212,
  "tf_epochs": 176,
  "log_frequency": 10,
  "N_u": 1000,
  "N_f": 14522
}

TensorFlow version: 2.3.0
Eager execution: True
GPU-accerelated: True

Training started
-- Starting Adam optimization --
tf_epoch =      0  elapsed = 00:00 (+00.1)  loss = 6.9438e-01  
tf_epoch =     10  elapsed = 00:01 (+01.5)  loss = 1.5213e+00  
tf_epoch =     20  elapsed = 00:03 (+01.5)  loss = 8.2432e-02  
tf_epoch =     30  elapsed = 00:04 (+01.5)  loss = 1.8151e-02  
tf_epoch =     40  elapsed = 00:06 (+01.5)  loss = 6.2335e-02  
tf_epoch =     50  elapsed = 00:08 (+01.5)  loss = 6.7877e-02  
tf_epoch =     60  elapsed = 00:09 (+01.5)  loss = 5.4808e-02  
tf_epoch =     70  elapsed = 00:11 (+01.5)  loss = 7.6326e-02  
tf_epoch =     80  elapsed = 00:12 (+01.5)  loss = 3.7294e-02  
tf_epoch =     90  elapsed = 00:14 (+01.6)  loss = 4.3996e-02  
tf_epoch =    100  elapsed = 00:16 (+01.6)  loss = 2.9346e-02  
tf_epoch =

[I 2021-08-04 06:58:17,642] Trial 1 finished with value: -0.13811232972554038 and parameters: {'nt_lr': 0.3083755998718507, 'nt_epochs': 212, 'tf_epochs': 176, 'n_units_l0': 45, 'n_units_l1': 45, 'n_units_l2': 45, 'n_units_l3': 45, 'n_units_l4': 45, 'n_units_l5': 45, 'n_units_l6': 45, 'n_units_l7': 45, 'n_units_l8': 45, 'adam_learning_rate': 0.019159392015737152}. Best is trial 0 with value: -0.08271048691230308.


...

Hyperparameters:
{
  "nt_lr": 0.29051784826360716,
  "nt_ncorr": 100,
  "nt_epochs": 183,
  "tf_epochs": 152,
  "log_frequency": 10,
  "N_u": 1000,
  "N_f": 14522
}

TensorFlow version: 2.3.0
Eager execution: True
GPU-accerelated: True

Training started
-- Starting Adam optimization --
tf_epoch =      0  elapsed = 00:00 (+00.2)  loss = 6.9438e-01  
tf_epoch =     10  elapsed = 00:01 (+01.5)  loss = 1.7335e+00  
tf_epoch =     20  elapsed = 00:03 (+01.6)  loss = 4.4537e-02  
tf_epoch =     30  elapsed = 00:05 (+01.6)  loss = 2.4044e-02  
tf_epoch =     40  elapsed = 00:06 (+01.6)  loss = 7.3680e-02  
tf_epoch =     50  elapsed = 00:08 (+01.6)  loss = 6.1416e-02  
tf_epoch =     60  elapsed = 00:10 (+01.6)  loss = 7.0395e-02  
tf_epoch =     70  elapsed = 00:11 (+01.6)  loss = 5.0059e-02  
tf_epoch =     80  elapsed = 00:13 (+01.6)  loss = 2.5187e-02  
tf_epoch =     90  elapsed = 00:15 (+01.6)  loss = 3.1627e-02  
tf_epoch =    100  elapsed = 00:16 (+01.6)  loss = 2.1125e-02  
tf_epoch 

[I 2021-08-04 07:14:08,668] Trial 13 finished with value: -0.08484673022747675 and parameters: {'nt_lr': 0.29051784826360716, 'nt_epochs': 183, 'tf_epochs': 152, 'n_units_l0': 45, 'n_units_l1': 45, 'n_units_l2': 45, 'n_units_l3': 45, 'n_units_l4': 45, 'n_units_l5': 45, 'n_units_l6': 45, 'n_units_l7': 45, 'n_units_l8': 45, 'adam_learning_rate': 0.019474998466284487}. Best is trial 12 with value: -0.08161519989238468.


Hyperparameters:
{
  "nt_lr": 0.3467140600836183,
  "nt_ncorr": 100,
  "nt_epochs": 183,
  "tf_epochs": 160,
  "log_frequency": 10,
  "N_u": 1000,
  "N_f": 14522
}

TensorFlow version: 2.3.0
Eager execution: True
GPU-accerelated: True

Training started
-- Starting Adam optimization --
tf_epoch =      0  elapsed = 00:00 (+00.1)  loss = 6.9438e-01  
tf_epoch =     10  elapsed = 00:01 (+01.5)  loss = 1.8860e+00  
tf_epoch =     20  elapsed = 00:03 (+01.5)  loss = 4.4205e-02  
tf_epoch =     30  elapsed = 00:04 (+01.5)  loss = 2.9268e-02  
tf_epoch =     40  elapsed = 00:06 (+01.5)  loss = 6.4276e-02  
tf_epoch =     50  elapsed = 00:08 (+01.6)  loss = 4.0488e-02  
tf_epoch =     60  elapsed = 00:09 (+01.6)  loss = 4.9575e-02  
tf_epoch =     70  elapsed = 00:11 (+01.6)  loss = 4.5403e-02  
tf_epoch =     80  elapsed = 00:12 (+01.6)  loss = 3.7979e-02  
tf_epoch =     90  elapsed = 00:14 (+01.5)  loss = 3.2509e-02  
tf_epoch =    100  elapsed = 00:16 (+01.6)  loss = 2.3305e-02  
tf_epoch =

[I 2021-08-04 07:15:12,730] Trial 14 finished with value: -0.08798972618922977 and parameters: {'nt_lr': 0.3467140600836183, 'nt_epochs': 183, 'tf_epochs': 160, 'n_units_l0': 45, 'n_units_l1': 45, 'n_units_l2': 45, 'n_units_l3': 45, 'n_units_l4': 45, 'n_units_l5': 45, 'n_units_l6': 45, 'n_units_l7': 45, 'n_units_l8': 45, 'adam_learning_rate': 0.019684505068875383}. Best is trial 12 with value: -0.08161519989238468.


Number of finished trials:  15
Best trial:
  Value:  -0.08161519989238468
  Params: 
    nt_lr: 0.27729275963241484
    nt_epochs: 189
    tf_epochs: 153
    n_units_l0: 45
    n_units_l1: 45
    n_units_l2: 45
    n_units_l3: 45
    n_units_l4: 45
    n_units_l5: 45
    n_units_l6: 45
    n_units_l7: 45
    n_units_l8: 45
    adam_learning_rate: 0.019470029017674015
--- 1189.2080872058868 seconds ---


In [19]:
fig = optuna.visualization.plot_contour(study, params=["nt_lr", 'adam_learning_rate', 'nt_epochs'])
fig.show()

In [14]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [26]:
import optuna

In [29]:
import pickle